In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 55202, done.
remote: Counting objects: 100% (680/680), done.
remote: Compressing objects: 100% (421/421), done.
remote: Total 55202 (delta 489), reused 263 (delta 259), pack-reused 54522 (from 4)
Receiving objects: 100% (55202/55202), 31.33 MiB | 29.59 MiB/s, done.
Resolving deltas: 100% (40728/40728), done.


In [2]:
!nvidia-smi

Mon Apr  7 04:29:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%cd ultralytics
!pip install -e '.[dev]'

/kaggle/working/ultralytics
Obtaining file:///kaggle/working/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 39.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 102.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 92.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 6.8 M

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.3/8062.4 GB disk)


In [23]:
%%writefile /kaggle/working/ultralytics/ultralytics/cfg/models/v8/conf.yaml
# Parameters
nc: 2 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [128, 128, 32, 3]] # 4
  - [-1, 1, Conv, [128, 3, 2]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [256, 256, 48, 3]] # 6
  - [-1, 1, Conv, [256, 3, 2]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [256, 256, 64, 3]] # 8
  - [-1, 1, SPPF, [256, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


Overwriting /kaggle/working/ultralytics/ultralytics/cfg/models/v8/conf.yaml


In [24]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO

if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/conf.yaml')
    model.info()

Overwriting train.py


In [25]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
conf summary: 302 layers, 2,218,182 parameters, 2,218,166 gradients, 7.4 GFLOPs


In [5]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("np_mynp")
version = project.version(5)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.0 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to np_mynp-5 in yolov8:: 100%|██████████| 5622/5622 [00:00<00:00, 6470.03it/s]


In [6]:
%%writefile /kaggle/working/ultralytics/np_mynp-5/data1.yaml
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/5
  version: 5
  workspace: buddetection
test: /kaggle/working/ultralytics/np_mynp-5/test/images
train: /kaggle/working/ultralytics/np_mynp-5/train/images
val: /kaggle/working/ultralytics/np_mynp-5/valid/images


Writing /kaggle/working/ultralytics/np_mynp-5/data1.yaml


In [26]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/conf.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640,optimizer='SGD',close_mosaic=10,)

Overwriting train.py


In [27]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/conf.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frame

In [21]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v8/yolov8.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)

Overwriting train.py


In [22]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

In [34]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v9/yolov9t.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)

Overwriting train.py


In [35]:
!python train.py

Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v9/yolov9t.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

In [36]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v10/yolov10n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)

Overwriting train.py


In [37]:
!python train.py

Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v10/yolov10n.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

In [46]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/11/yolo11.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)

Overwriting train.py


In [47]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/11/yolo11.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

In [48]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/12/yolo12.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)

Overwriting train.py


In [49]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/12/yolo12.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

In [50]:
!ls

CITATION.cff	 examples    pyproject.toml   tests		    yolo11n.pt
CONTRIBUTING.md  LICENSE     README.md	      train.py
docker		 mkdocs.yml  README.zh-CN.md  ultralytics
docs		 np_mynp-5   runs	      ultralytics.egg-info


In [52]:

import shutil

# Zip the 'runs' directory
shutil.make_archive("runs_backup", 'zip', "runs")

print("Zipping complete! File saved as runs_backup.zip")


Zipping complete! File saved as runs_backup.zip


In [7]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v5/yolov5.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    #results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640,optimizer='SGD',close_mosaic=10,)
    model.info()

Writing train.py


In [8]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
YOLOv5 summary: 153 layers, 2,654,816 parameters, 2,654,800 gradients, 7.8 GFLOPs


In [10]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v5/yolov5.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)
    #model.info()

Overwriting train.py


In [ ]:
!python train.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.102 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v5/yolov5.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

In [8]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v6/yolov6n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    #results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)
    model.info()

Writing train.py


In [9]:
!python train.py

YOLOv6n summary: 121 layers, 4,500,080 parameters, 4,500,064 gradients, 13.1 GFLOPs


In [7]:
%%writefile train.py

import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO


if __name__ == '__main__':
    model = YOLO('ultralytics/cfg/models/v6/yolov6n.yaml')
    # model.load('yolov8n.pt') # loading pretrain weights
    results=model.train(data="/kaggle/working/ultralytics/np_mynp-5/data1.yaml", epochs=100, batch=16, imgsz=640)
    #model.info()

Writing train.py


In [8]:
!python train.py

Ultralytics 8.3.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v6/yolov6n.yaml, data=/kaggle/working/ultralytics/np_mynp-5/data1.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop

In [9]:
import shutil

# Zip the 'runs' directory
shutil.make_archive("runs_backup_v6", 'zip', "runs")

print("Zipping complete! File saved as runs_backup.zip")

Zipping complete! File saved as runs_backup.zip
